In [ ]:
import sqlite3
# Connect (creates file if not exists)
conn = sqlite3.connect("hospital.db")
conn.execute("PRAGMA foreign_keys = ON;")
print("Created DB successfully!")

In [ ]:
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS doctors (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  specialty TEXT NOT NULL,
  department TEXT NOT NULL,
  location TEXT NOT NULL
);
""")
conn.commit()
conn.close()
print("Created doctors Table")

In [ ]:
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS labs (
  id INTEGER PRIMARY KEY,
  name TEXT NOT NULL,
  location TEXT NOT NULL
);
""")
conn.commit()
conn.close()
print("Created labs Table")

In [ ]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS availability (
  id INTEGER PRIMARY KEY,
  resource_type TEXT NOT NULL CHECK (resource_type IN ('doctor','lab','service')),
  resource_id INTEGER NOT NULL,
  slot_start TEXT NOT NULL,
  slot_end TEXT NOT NULL,
  is_available INTEGER NOT NULL CHECK (is_available IN (0,1))
);
""")
conn.commit()
conn.close()
print("Created availability Table")

In [ ]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS appointments (
  id INTEGER PRIMARY KEY,
  user_id TEXT NOT NULL,
  kind TEXT NOT NULL,
  resource_id INTEGER,
  resource_type TEXT,
  requested_slot TEXT,
  booked_slot TEXT,
  status TEXT NOT NULL,
  suggested_alternatives TEXT,
  created_at TEXT NOT NULL DEFAULT (datetime('now'))
);
""")
conn.commit()
conn.close()
print("Created appointments Table")

In [ ]:
#Fetch All Records
import sqlite3
conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM doctors")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

#Fetch a Single Record
import sqlite3
conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM doctors WHERE id = ?", (1,))
row = cursor.fetchone()
print(row)

#Fetch Records as Dictionary
import sqlite3
def dict_factory(cursor, row):
    return {col[0]: row[idx] for idx, col in enumerate(cursor.description)}
conn = sqlite3.connect("hospital.db")
conn.row_factory = dict_factory  # <-- THIS LINE
cursor = conn.cursor()
cursor.execute("SELECT * FROM doctors")
rows = cursor.fetchall()
for row in rows:
    print(row)   # prints JSON-style dict
conn.close()

#Fetch with Conditions
import sqlite3
conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()
cursor.execute("""
SELECT * FROM doctors WHERE specialty = ?
""", ("Cardiology",))
rows = cursor.fetchall()
print(rows)

#Get Availability for a Doctor
import sqlite3
conn = sqlite3.connect("hospital.db")
cursor = conn.cursor()
cursor.execute("""
SELECT * FROM availability
WHERE resource_type = 'doctor' 
AND resource_id = ?
""", (1,))
slots = cursor.fetchall()
print(slots)

#BEST PRACTICE: Use With Block (Auto-close)
import sqlite3
with sqlite3.connect("hospital.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM appointments")
    rows = cursor.fetchall()
    print(rows)


In [ ]:
# import sqlite3
# conn = sqlite3.connect("hospital.db")
# cursor = conn.cursor()
# doctors = [
#     ("Dr. Sarah Paul", "Orthopedics", "Bone Center", "Building B"),
#     ("Dr. Amit Rao", "Neurology", "Brain Center", "Building C"),
#     ("Dr. Leena Patel", "Dermatology", "Skin Clinic", "Building D")
# ]
# cursor.executemany("""
# INSERT INTO doctors (name, specialty, department, location)
# VALUES (?, ?, ?, ?)
# """, doctors)
# conn.commit()
# conn.close()
# print("Many doctors inserted!")

# import sqlite3
# conn = sqlite3.connect("hospital.db")
# cursor = conn.cursor()
# cursor.execute("""
# INSERT INTO availability (
#   resource_type, resource_id, slot_start, slot_end, is_available
# )
# VALUES (?, ?, ?, ?, ?)
# """, ("doctor", 1, "2025-01-10 10:00", "2025-01-10 10:30", 1))
# conn.commit()
# conn.close()
# print("Availability inserted!")

# import sqlite3
# conn = sqlite3.connect("hospital.db")
# cursor = conn.cursor()
# cursor.execute("""
# INSERT INTO appointments (
#   user_id, kind, resource_id, resource_type,
#   requested_slot, booked_slot, status
# )
# VALUES (?, ?, ?, ?, ?, ?, ?)
# """, (
#     "user001",
#     "doctor",
#     1,
#     "doctor",
#     "2025-01-10 10:00",
#     "2025-01-10 10:00",
#     "pending"
# ))
# conn.commit()
# conn.close()
# print("Appointment inserted!")

# import sqlite3
# conn = sqlite3.connect("hospital.db")
# cursor = conn.cursor()
# cursor.execute("""
# INSERT INTO labs (name, location)
# VALUES (?, ?)
# """, ("Microbiology Lab", "Building C"))
# cursor.execute("""
# INSERT INTO labs (name, location)
# VALUES (?, ?)
# """, ("Radiology Lab", "Building D"))
# conn.commit()
# conn.close()
# print("Lab records added!")




In [ ]:
import sqlite3
with sqlite3.connect("hospital.db") as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM labs")
    rows = cursor.fetchall()
    print(rows)

In [ ]:
import json
from my_llm_client import llm
from datetime import datetime, timedelta

def extract_entities_and_update_state(state: dict) -> dict:
    query = state["query"]

    # Ask LLM to extract structured entities
    raw_entities = llm.invoke({
        "prompt": f"""
        Extract entities from this appointment request as JSON.
        Keys: type (doctor/lab/disease/service), doctor_name, specialty, department,
              test, disease, service, date, time, location.
        Rules:
        - If no date is mentioned, set "date" to "tomorrow".
        - If no time is mentioned, set "time" to "09:00".
        Input: "{query}"
        """
    })

    try:
        entities = json.loads(raw_entities)
    except Exception:
        entities = {}

    # Handle date
    date_str = entities.get("date")
    if not date_str or date_str.lower() == "tomorrow":
        parsed_date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
    else:
        try:
            parsed_date = datetime.strptime(date_str, "%Y-%m-%d").strftime("%Y-%m-%d")
        except Exception:
            parsed_date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

    # Handle time
    time_str = entities.get("time") or "09:00"

    # Update state with extracted values
    state.update({
        "kind": entities.get("type"),
        "doctor_name": entities.get("doctor_name"),
        "specialty": entities.get("specialty"),
        "department": entities.get("department"),
        "test": entities.get("test"),
        "disease": entities.get("disease"),
        "service": entities.get("service"),
        "date": parsed_date,
        "time": time_str,
        "location": entities.get("location"),
        # Combine date and time into one string for DB queries
        "preferred_start": f"{parsed_date} {time_str}"
    })

    return state

In [ ]:
import sqlite3, json

def find_matching_slots(resource_type, resource_id, preferred_start):
    conn = sqlite3.connect("triage.db")
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    # Check if preferred slot exists
    c.execute("""
        SELECT * FROM availability
        WHERE resource_type=? AND resource_id=? AND slot_start=?
    """, (resource_type, resource_id, preferred_start))
    preferred = c.fetchone()

    # Get up to 5 alternative slots
    c.execute("""
        SELECT * FROM availability
        WHERE resource_type=? AND resource_id=? AND is_available=1
        ORDER BY slot_start ASC
        LIMIT 5
    """, (resource_type, resource_id))
    alternatives = c.fetchall()

    conn.close()
    return preferred, alternatives

def suggest_alternatives_json(alts):
    return json.dumps([dict(r) for r in alts], indent=2)

def book_slot(resource_type, resource_id, slot_start):
    conn = sqlite3.connect("triage.db")
    c = conn.cursor()
    c.execute("""
        UPDATE availability
        SET is_available=0
        WHERE resource_type=? AND resource_id=? AND slot_start=? AND is_available=1
    """, (resource_type, resource_id, slot_start))
    ok = (c.rowcount == 1)
    conn.commit()
    conn.close()
    return ok